In [32]:
%load_ext autoreload
%autoreload 2
import gym
import os
import numpy as np
import random
import torch
from experiment import rollout, ReplayBuffer, Trajectory, load_model, save_model, Behavior
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
env = gym.make('CartPole-v1')

In [34]:
rb = ReplayBuffer(max_size=500, last_few=200)

# Random rollout
trajectories, avg_reward = rollout(episodes=1000, env=env, render=False)
rb.add(trajectories)

print(f"Average Episode Reward: {avg_reward}")

Average Episode Reward: 22.312


In [35]:
loss_object = torch.nn.BCEWithLogitsLoss().to(device)

d = env.observation_space.shape[0]
    
model = Behavior(input_shape=d+2, num_actions=1).to(device)
optimizer = torch.optim.Adam(model.parameters())

In [36]:
batch_size = 1024

In [44]:
def train_step(inputs, targets):
    optimizer.zero_grad()    
    predictions = model(inputs)
    targets = targets.float()
    loss = loss_object(predictions, targets.unsqueeze(1))
    
    loss.backward()
    optimizer.step()
    
    return loss

def action_fn(model, inputs, sample_action=True):
    action_probs = model(inputs)
    action_probs = torch.sigmoid(action_probs)

    if sample_action:
        m = torch.distributions.bernoulli.Bernoulli(probs=action_probs)            
        action = int(m.sample().squeeze().cpu().numpy())
    else:
        action = int(np.round(action_probs.detach().squeeze().numpy()))
    return action
    

In [45]:
loss_sum = 0
loss_count = 0

In [ ]:
epochs = 1000000
epoch, model, optimizer, loss = load_model('cart_pole', model, optimizer, device, train=True)

for i in range(epoch, epochs+epoch):
    x, y = rb.sample(batch_size, device)    
    loss = train_step(x, y)
    loss_sum += loss
    loss_count += 1
    
    if i % 100 == 0:        
        trajectories, mean_reward = rollout(100, env=env, model=model, sample_action=True, replay_buffer=rb, 
                              device=device, action_fn=action_fn)
        rb.add(trajectories)
        
        print(f"Average Episode Reward: {mean_reward}")        

    if i % 200 == 0:
        print(f'i: {i}, Loss: {loss_sum/loss_count}') #'\t Accuracy: {accuracy_m.result()}')
        save_model('cart_pole', i, model, optimizer, loss)

In [48]:
rb.sample_command()

(51.115, 59.018343913480535)

In [52]:
cmd = (500, 500) #rb.sample_command()
env = gym.make('CartPole-v1')
e, model, _, l = load_model('cart_pole', train=False, model=model, optimizer=optimizer, device=device)
print(f"Loaded model at epoch: {e} with loss {l}")
_, mean_reward = rollout(episodes=3, env=env, model=model, sample_action=False, 
                      replay_buffer=rb, render=True, device=device, action_fn=action_fn)


print(f"Average Episode Reward: {mean_reward}")

Existing model found. Loading.
Loaded model at epoch: 400 with loss 0.6721066832542419
Average Episode Reward: 52.333333333333336
